# Different strategy for different variables(Numerical & Categorical) with Scikit-Learn

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [5]:
train=pd.read_csv(r"E:\1biplav\data\Dataset\House Price Prediction\train.csv")
test=pd.read_csv(r"E:\1biplav\data\Dataset\House Price Prediction\test.csv")
print("shaope of train data=",train.shape)
print("shape of test data=",test.shape)

shaope of train data= (1460, 81)
shape of test data= (1459, 80)


In [21]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [62]:
X_train=train.drop(columns="SalePrice",axis=1)
y_train=train["SalePrice"]
X_test=test.copy
print("Shape of X_train=",X_train.shape)
print("Shape of y_train=",y_train.shape)


Shape of X_train= (1460, 80)
Shape of y_train= (1460,)


# Missing Values imputation 

In [78]:
isnull_sum=X_train.isnull().sum()
isnull_sum

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
Length: 80, dtype: int64

In [64]:
#finding the numerical variale which have missing values
num_vars=X_train.select_dtypes(include=["int64","float64"]).columns
num_vars_miss=[var for var in num_vars if  isnull_sum[var]>0]
num_vars_miss

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [65]:
#finding categorical variable which have missing values 
cat_var=X_train.select_dtypes(include=["O"]).columns
cat_var_miss=[var for var in cat_var if isnull_sum[var]>0]
cat_var_miss

['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [66]:
"""Let's create a pipeline.. which value to put in what will first look in the data then
Will make the data a value and a variable"""
#create pipiline 
num_var_mean=["LotFrontage"] #put mean
num_var_median=["'MasVnrArea', 'GarageYrBlt'"] #put median
cat_vars_mode=['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu']
cat_vars_missing=['GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [67]:
#create pipeline one by one  using imputer
num_var_mean_imputer = Pipeline(steps=[("imputer", SimpleImputer(strategy="mean"))])
num_var_median_imputer = Pipeline(steps=[("imputer", SimpleImputer(strategy="median"))])
cat_vars_mode_imputer = Pipeline(steps=[("imputer", SimpleImputer(strategy="most_frequent"))])
cat_vars_missing_imputer = Pipeline(steps=[("imputer", SimpleImputer(strategy="constant", fill_value="missing"))])

In [79]:
preprocessor = ColumnTransformer(transformers=[("mean_imputer", num_var_mean_imputer, num_var_mean),
                                ("median_imputer", num_var_median_imputer, num_var_median),
                               ("mode_imputer", cat_vars_mode_imputer, cat_vars_mode),
                               ("missing_imputer", cat_vars_missing_imputer,cat_vars_missing)])
preprocessor

ColumnTransformer(transformers=[('mean_imputer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['LotFrontage']),
                                ('median_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ["'MasVnrArea', 'GarageYrBlt'"]),
                                ('mode_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='mode'))]),
                                 ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond',
                                  'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2', 'Electrical',
                                  'FireplaceQu']),
                                ('missing_imputer',
                                 Pipeline(st

In [80]:
preprocessor.fit(X_train)

ValueError: A given column is not a column of the dataframe